In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import re
import numpy as np

In [12]:
#读入数据，并且用beautsoup去除杂七杂八的网页标签，review返回一个list，list反映content的各个单词

In [2]:
labeltrain = pd.read_csv("labeledTrainData.tsv",header=0, delimiter="\t", quoting=3)
unlabeltrain = pd.read_csv("unlabeledTrainData.tsv",header=0, delimiter="\t", quoting=3)
testdata = pd.read_csv("testData.tsv",header=0, delimiter="\t", quoting=3)

In [3]:
def getwords(review,remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    
    if remove_stopwords:
        document = open('/Users/houxiaohui/anaconda/lib/python2.7/site-packages/nltk_data/corpora/stopwords/english')
        stopwords = []
        for word in document:
            stopwords.append(word)
        stops = set(stopwords)
        words = [w for w in words if not w in stops]
    return words

In [4]:
y = labeltrain.sentiment

In [17]:
#因为要使用TF-IDF，因此要有一个包含了所有单词的全文本

In [6]:
train_words = []
for i in range(len(labeltrain)):
    review = labeltrain['review'][i]
    wordlist = getwords(review)
    train_words.append(" ".join(wordlist))

/Users/houxiaohui/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
test_words = []
for i in range(len(testdata)):
    review = testdata['review'][i]
    wordlist = getwords(review)
    test_words.append(" ".join(wordlist))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIV

In [19]:
tfv = TFIV(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word',\
           token_pattern=r'\w{1,}', ngram_range=(1, 2), use_idf=1,smooth_idf=1,\
           sublinear_tf=1, stop_words = 'english')

In [20]:
x_all = train_words+test_words

In [21]:
x_all = tfv.fit_transform(x_all)
x = x_all[:len(train_words)]
x_test = x_all[len(train_words):]

In [22]:
from sklearn.naive_bayes import MultinomialNB as MNB
nb = MNB()
nb.fit(x,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
from sklearn.cross_validation import cross_val_score
import numpy as np

print "5折交叉验证得分: ",np.mean(cross_val_score(nb,x,y,cv=5,scoring='roc_auc'))

5折交叉验证得分:  0.948345504


In [40]:
predict = nb.predict(x_test)

In [41]:
submission = pd.DataFrame({"id":testdata['id'],"sentiment":predict})

In [42]:
submission.to_csv("submission.csv",index=False)

In [43]:
#利用TF-IDF和多项贝叶斯在test集上得到的结果是0.85724。很不解，朴素贝叶斯的overfit很小，但五折交叉验证明显优于测试集

## 利用word2vec进行模型预测 

https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors

在TF-IDF中，为了更好地计算每个词的重要性，把停用词去掉，但是word2vec中最好不要去掉。因为word2vec的词语向量h会会根据停用词算出。TF-IDF中喂给算法的是单词，但在word2vec中最好的喂给算法的是一个句子。但是怎么样才能中断句子呢？科学家们认为，即使用标点符号，大小写等也是不够准确的，python有一个nltk包可以帮助我们做到区分句子

In [25]:
#安装nltk包，并且下载相关语料包
import nltk.data

In [26]:
#nltk.download()

In [27]:
#加载标点符号分词器
tokenizer = nltk.data.load('/Users/houxiaohui/anaconda/lib/python2.7/site-packages/tokenizers/punkt/english.pickle')

In [28]:
#定义一个函数划分句子
def review_to_sentences(review, tokenizer,remove_stopwords=False):
    #用NLTK分词器把段落划分成为句子
    review = review.decode("utf-8")    #把review解码，否则报错
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    #sentences[i]是一个包含了该句话所有单词组成的list
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(getwords( raw_sentence,remove_stopwords))
    return sentences

In [29]:
sentences = []
print "把labeltrain的句子划分出来"
for review in labeltrain['review']:
    sentences+= review_to_sentences(review,tokenizer)
print "把unlabeltrain的句子划分出来"
for review in unlabeltrain['review']:
    sentences+=review_to_sentences(review,tokenizer)

把labeltrain的句子划分出来


/Users/houxiaohui/anaconda/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Users/houxiaohui/anaconda/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


把unlabeltrain的句子划分出来


/Users/houxiaohui/anaconda/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Users/houxiaohui/anaconda/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful 

In [30]:
print sentences[1]

[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


In [31]:
#Import the built-in logging module and configure it so that Word2Vec 
#creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [33]:
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3
from gensim.models import word2vec
print "training model"
model = word2vec.Word2Vec(sentences, workers=num_workers,size=num_features,min_count=min_word_count,\
                         window =context, sample=downsampling)

training model


In [34]:
#如果不打算更改参数，init_sims可以令到模型更加有效
model.init_sims(replace=True)
#保存模型
model_name = "300features_40windows_10context"
model.save(model_name)

In [35]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [37]:
model.syn0.shape

(16490, 300)

In [87]:
#把保存的model加载出来
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40windows_10context")

In [94]:
#flower 300维向量表示
print model["flower"].shape
print model['flower']

(300,)
[-0.06180373 -0.03055314 -0.02788739 -0.00303928  0.05845078  0.02961411
  0.05386321  0.01109022  0.05256153 -0.0449115   0.02647414 -0.05939076
  0.05391535  0.05881985 -0.07142239 -0.08370717  0.0069117  -0.06156367
 -0.02996042 -0.02917108 -0.05412992  0.00463001  0.05070795  0.04969992
 -0.02298744  0.04569796 -0.03031595 -0.04911005 -0.02743537  0.07245769
 -0.00487353 -0.01499925 -0.0151846   0.02670506 -0.01087801  0.06588218
  0.00762568 -0.01244507  0.07263737  0.00819124  0.00708139 -0.00480718
 -0.05014724 -0.0069901   0.00170102  0.0041009  -0.01330999 -0.04403951
 -0.01806104  0.07172135  0.02140717  0.02385642  0.04330316  0.01740825
  0.01324386  0.07636381  0.00933965  0.0473394  -0.11823727  0.02022422
 -0.05764703  0.01486576  0.06008713  0.00676664 -0.05963673  0.13316357
  0.019418    0.01395135  0.10422439 -0.05284138 -0.05169625  0.01932217
  0.02650642 -0.00288717 -0.01929533 -0.04086848 -0.03387094  0.04883172
 -0.03066172  0.06306476  0.02690616  0.0253

现在单词的维度统一了，但是不同的review的向量还没有得出，以下代码要实现的是把单词每个维度取平均数，赋给每个review，弄成一个矩阵
此时，移除停用词，否则会加入太多的噪声

In [95]:
def makeFeatureVec(words, model, num_features):
    #该方程算出所有单词的300维中的每一维的平均向量
    # Function to average all of the word vectors in a given paragraph
    featureVec = np.zeros((num_features),dtype=float)
    nwords = 0
    
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords+=1
            featureVec =np.add(featureVec,model[word])
    
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [123]:
def getAvgFeatureVecs(reviews, model, num_features):
    #Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    #计算每一个review的平均向量，返回一个2D的矩阵
    
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype=float)
    
    for review in reviews:
        if counter%1000 == 0:
            print "Review %d of %d"%(counter, len(reviews))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter+1
    return reviewFeatureVecs

In [125]:
print '''为了防止噪声干扰，移除停用词'''
clean_train_reviews = []
for review in labeltrain['review']:
    clean_train_reviews.append(getwords(review, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_reviews,model,num_features)

clean_test_reviews = []
for review in testdata['review']:
    clean_test_reviews.append(getwords(review, remove_stopwords=True))
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

为了防止噪声干扰，移除停用词
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000

现在得到了每个review在300维的向量，放到随机森林上进行建模

In [95]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

forest = forest.fit(trainDataVecs, y)
y_hat = forest.predict(testDataVecs)

output = pd.DataFrame( data={"id":testdata["id"], "sentiment":y_hat} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

用随机森林算法拟合训练集


In [96]:
#word2vec加上随机森林在测试集上的LB是0.82940

既然Word2Vec能把单词变成向量，那么可以通过向量进行聚类，看看哪些单词是一堆的

In [130]:
from sklearn.cluster import KMeans
import time

In [133]:
#每5个单词一类
start = time.time()
wordVectors = model.syn0
num_clusters = wordVectors.shape[0]/5

km = KMeans(n_clusters=num_clusters)
idx = km.fit_predict(wordVectors)
end = time.time()
print "利用kmeans聚类单词所用的时间： 分钟", int(end-start)/60

利用kmeans聚类单词所用的时间：  1456.46670794


In [138]:
#现在单词向量被分成了很多类，但是人是看不懂向量的，要把向量转成单词
allwords = model.index2word
for cluster in range(10):
    words = []
    for i in range(len(wordVectors)):
        if idx[i] == cluster:
            words.append(allwords[i])
    print words

[u'iceberg']
[u'hayward']
[u'layers', u'revelations', u'explanations']
[u'roaming', u'roam']
[u'blade', u'runner', u'blazing', u'cape', u'claw', u'cobra', u'raider']
[u'hysteria', u'oppression', u'liberation', u'persecution', u'probable', u'secular']
[u'whit', u'gibbs', u'weber']
[u'swords', u'armor', u'elephants', u'barf']
[u'whipped', u'nailed', u'pinned', u'battered']
[u'pompous', u'smug', u'smarmy', u'dishonest', u'humorless', u'overplayed', u'unintelligent']


用每个向量去做随机森林太耗费时间，既然已经聚类出来了，可以用每个单词的聚类作为特征，喂给随机森林

In [139]:
word_centroid_map = dict(zip(model.index2word,idx))

In [80]:
def create_bag_of_centroids(review,):
    #返回一个数列，第i行代表第i类有多少个单词
    bag_of_centroids = np.zeros(num_clusters,dtype=float)
    for word in review:
        if word_centroid_map.has_key(word):
            index = word_centroid_map[word]
            bag_of_centroids[index]+=1
    return bag_of_centroids

In [89]:
train_centroids = np.zeros((len(labeltrain['review']),num_clusters),dtype=float)
for i in range(len(clean_train_reviews)):
    review = clean_train_reviews[i]
    train_centroids[i] = create_bag_of_centroids(review)
test_centroids = np.zeros((len(testdata['review']), num_clusters),dtype=float)
for i in range(len(clean_test_reviews)):
    review = clean_test_reviews[i]
    test_centroids[i] = create_bag_of_centroids(review)

In [94]:
forest = RandomForestClassifier(n_estimators = 100)
print "Fitting a random forest to labeled training data..."
forest = forest.fit(train_centroids,y)
result = forest.predict(test_centroids)
# Write the test results 
#output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
#output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


## 改进算法

### 改进TF-IDF

In [97]:
#上面的word2vec返回的单词go和went是不一样的，nltk中提供了提取单词词根的库

In [98]:
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

In [101]:
def getwords(review,remove_stopwords=False):
    review_text = BeautifulSoup(review).get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    
    if remove_stopwords:
        document = open('/Users/houxiaohui/anaconda/lib/python2.7/site-packages/nltk_data/corpora/stopwords/english')
        stopwords = []
        for word in document:
            stopwords.append(word)
        stops = set(stopwords)
        words = [w for w in words if not w in stops]
    #提取特征词
    stemmerWords = []
    for word in words:
        stemmerWords.append(english_stemmer.stem(word))
    return stemmerWords

In [102]:
print '''为了防止噪声干扰，移除停用词'''
clean_train_reviews = []
for review in labeltrain['review']:
    clean_train_reviews.append(getwords(review, remove_stopwords=True))
    
clean_test_reviews = []
for review in testdata['review']:
    clean_test_reviews.append(getwords(review, remove_stopwords=True))

clean_unlabeltrain_reviews = []
for review in unlabeltrain['review']:
    clean_unlabeltrain_reviews.append(getwords(review, remove_stopwords=True))

为了防止噪声干扰，移除停用词


In [115]:
cleaned_train_reviews = []
for reviewlist in clean_train_reviews:
    cleaned_train_reviews.append(" ".join(reviewlist))
    
cleaned_unlabeltrain_reviews = []
for reviewlist in clean_unlabeltrain_reviews:
    cleaned_unlabeltrain_reviews.append(" ".join(reviewlist))
    
cleaned_test_reviews = []
for reviewlist in clean_test_reviews:
    cleaned_test_reviews.append(" ".join(reviewlist))

In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
print "vectorizing words"
vectorizer = TfidfVectorizer( min_df=2, ngram_range = ( 1, 4 ),
                              sublinear_tf = True )
vectorizer = vectorizer.fit(cleaned_train_reviews + cleaned_unlabeltrain_reviews)
train_data_features = vectorizer.transform(cleaned_train_reviews)
test_data_features = vectorizer.transform(cleaned_test_reviews)

vectorizing words


In [117]:
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
fselect = SelectKBest(chi2 , k=70000)
train_data_features = fselect.fit_transform(train_data_features, y)
test_data_features = fselect.transform(test_data_features)

In [121]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
model1 = MultinomialNB(alpha=0.0005)
model1.fit( train_data_features, y)

model2 = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
model2.fit( train_data_features,y )

p1 = model1.predict( test_data_features )
p2 = model2.predict( test_data_features )

print "Writing results..."

output = pd.DataFrame( data = { "id": testdata["id"], "sentiment": p1 } )
output.to_csv("TFIDFmultiNB.csv", index = False, quoting = 3 )

output = pd.DataFrame( data = { "id": testdata["id"], "sentiment": p2 } )
output.to_csv("TFIDFSGBClassifier.csv", index = False, quoting = 3 )

Writing results...


## 改进word2vec的参数

word2vec_AverageVectors在做review向量化的时候，把每个单词的每个维度的向量相加
这种方式太简单粗暴
之前的Word2vecmodel，最小计数单词是40个，这个要求太严苛了
num_features = 300
min_word_count = 5
num_workers = 4
context = 10
downsampling = 1e-3
from gensim.models import word2vec
print "training model"
model = word2vec.Word2Vec(sentences, workers=num_workers,size=num_features,min_count=min_word_count,\
                         window =context, sample=downsampling)

## 加入doc2vec算法

In [5]:
#word2vec在训练在像微博这样的140个词语以内的短文是有效的，但是当训练长文的时候就需要doc2vec

In [6]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence

In [10]:
def getCleanWords(data,remove_stopwords):
    clean_reviews = []
    for review in data['review']:
        clean_reviews.append(getwords(review,remove_stopwords))
    
    #doc2vec训练句子的时候要有一个label
    labelized = []
    for i, id_label in enumerate(data['id']):
        labelized.append(LabeledSentence(clean_reviews[i], [id_label]))
    return labelized

In [27]:
labelizedLableTrain = getCleanWords(labeltrain,remove_stopwords=True)
labelizedUnlableTrain = getCleanWords(unlabeltrain,remove_stopwords=True)
labelizedTestData = getCleanWords(testdata,remove_stopwords=True)

In [28]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [29]:
num_features = 2000
min_word_count = 10
num_workers = 4
context = 10
downsampling = 1e-3

In [30]:
print "trainging data with Doc2Vec"
model_dm = Doc2Vec(min_count=min_word_count, window=context, size=num_features, \
                           sample=downsampling, workers=num_workers)
model_dbow = Doc2Vec(min_count=min_word_count, window=context, size=num_features, 
                             sample=downsampling, workers=num_workers, dm=0)

trainging data with Doc2Vec


In [34]:
all_reviews = np.concatenate((labelizedLableTrain + labelizedUnlableTrain + labelizedTestData))

In [35]:
model_dm.build_vocab(all_reviews)
model_dbow.build_vocab(all_reviews)

AttributeError: 'list' object has no attribute 'words'

In [ ]:
import random
for epoch in range(10):
    perm = np.random.permutation(all_reviews.shape[0])
    model_dm.train(all_reviews[perm])
    model_dbow.train(all_reviews[perm])

In [ ]:
def getFeatureVecs(reviews, model, num_features):
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    counter = -1
    
    for review in reviews:
        counter = counter + 1
        try:
            reviewFeatureVecs[counter] = np.array(model[review.labels[0]]).reshape((1, num_features))
        except:
            continue
    return reviewFeatureVecs

In [ ]:
getFeatureVecs(labelizedLableTrain, model_dm, n_dim)

# 总结：
采用了三种方法对单词的向量进行处理，分别是TF-IDF, Word2Vec， Doc2Vec
TF-IDF利用了单词出现的频率
Word2Vec是和Doc2Vec是利用神经网络，具体原理不懂
### Word2Vec
1. 利用tokenize把review切成句子
2. 每句单词进行清洗，包括去除奇奇怪怪的字符，只保留英文单词，停用词有选择去除或者不去除
3. 切成句子之后的单词要分开，喂给word2vec的是一个list，list中的每一项也是一个list，里面都是清洗好的单词
4. word2vec.Word2Vec（sentences） 把每个单词转化为向量就能够当做特征进行分类、降维、聚类

### Doc2Vec
1. 同样clean word,但是要给每个sentences一个label
2. 有dm和dow两种模型
3. 要循环几次训练